In [0]:
CREATE OR REPLACE FUNCTION dbdemos_vishesh.contest_mock.get_contest_analytics
(
    sport_name STRING COMMENT 'Name of the sport (e.g., cricket, football, kabaddi)',
    contest_year INT COMMENT 'Year to filter contests (extracted from contest_create_ts)'
) 
RETURNS STRUCT<
    most_popular_contest_type: STRING,
    average_prize_pool: DOUBLE,
    total_contests: BIGINT
>
COMMENT 'Returns the most popular contest type and average prize pool for a given sport and year'
LANGUAGE SQL
RETURN 
    SELECT STRUCT(
        contest_analytics.most_popular_type,
        contest_analytics.avg_prize_pool,
        contest_analytics.contest_count
    )
    FROM (
        SELECT 
            FIRST(contest_type) as most_popular_type,
            AVG(prize_pool) as avg_prize_pool,
            COUNT(*) as contest_count
        FROM (
            SELECT 
                contest_type,
                prize_pool,
                ROW_NUMBER() OVER (ORDER BY type_count DESC, contest_type) as rn
            FROM (
                SELECT 
                    contest_type,
                    prize_pool,
                    COUNT(*) OVER (PARTITION BY contest_type) as type_count
                FROM dbdemos_vishesh.contest_mock.contests
                WHERE LOWER(contest_sport) = LOWER(sport_name)
                  AND YEAR(contest_create_ts) = contest_year
            ) type_counts
        ) ranked_types
        WHERE rn = 1
        GROUP BY contest_type
        
        UNION ALL
        
        SELECT 
            NULL as most_popular_type,
            AVG(prize_pool) as avg_prize_pool, 
            COUNT(*) as contest_count
        FROM dbdemos_vishesh.contest_mock.contests
        WHERE LOWER(contest_sport) = LOWER(sport_name)
          AND YEAR(contest_create_ts) = contest_year
    ) contest_analytics
    WHERE most_popular_type IS NOT NULL
    LIMIT 1;
